#Marim Ashraf Elsayed Mahmoud Amer

# Reddit Fake Post Detection

## Input

#### Text column

## Output

#### User needs to know the information is fake or not

## Data mining function

#### Classification and prediction because we need to know if the information is fake or not. 

## The challenges 

#### Cleaning data and removing all tags, special characters, ASCII, and white spaces from the text.

## Impact

#### We will know fake information.

## Steps

#### 1- Reading data and display data.
#### 2- Cleaning data and preprocessing data.
#### 3- Modeling and Splitting data
#### 4- Extract numerical features and categorical Features.
#### 5- Trials vectorizer analyzer="word".
#### 6- Trials vectorizer analyzer="char"



## Protocol 
#### I intend to use hold out method.


#**The best solution**

##Trials vectorizer analyzer="word"

**Trail_3**

Search: Grid Search.

Classifier: Logistic Classifier.

Public score: 0.83771

Private Score: 0.83929




#Import Libraries

In [ ]:
pip install scikit-optimize

     |████████████████████████████████| 100 kB 3.6 MB/s 


In [ ]:
#import liberaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



import re
import pickle
import sklearn
import nltk
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import train_test_split, PredefinedSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor

#Reading data

In [ ]:
#Read csv file
df = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/DM/Project_3/xy_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/DM/Project_3/x_test.csv")

In [ ]:
#Display raining data set
df

,id,text,label
0,265723,A group of friends began to volunteer at a hom...,0
1,284269,British Prime Minister @Theresa_May on Nerve A...,0
2,207715,"In 1961, Goodyear released a kit that allows P...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0
...,...,...,...
59995,70046,Finish Sniper Simo H盲yh盲 during the invasion o...,0
59996,189377,Nigerian Prince Scam took $110K from Kansas ma...,1
59997,93486,Is It Safe To Smoke Marijuana During Pregnancy...,0
59998,140950,Julius Caesar upon realizing that everyone in ...,0


In [ ]:
#Uniques values ​​and their iteration
df['label'].value_counts()

0    32172
1    27596
2      232
Name: label, dtype: int64

In [ ]:
#Drop all lables = 2 as there is a noise values
df.drop(df[df['label'] == 2].index, inplace = True)

In [ ]:
#Check uniques values ​​and their iteration
df['label'].value_counts()

0    32172
1    27596
Name: label, dtype: int64

#Cleaning and Preprocessing

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words("english"))

#Embedded need to know the meaning of the sentence
def clean_text(text):
    """ steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)  #White Space
    RE_TAGS = re.compile(r"<[^>]+>")    #Tags
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)    #ASCII
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE) #Single charachter between two spaces

    #Replace White Space, Tags, ASCII and Single charachter between two spaces with single space
    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    #Tokanization
    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    words_filtered = [
        stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
    ]

    text_clean = " ".join(words_filtered)
    return text_clean

In [ ]:
df

,id,text,label
0,265723,A group of friends began to volunteer at a hom...,0
1,284269,British Prime Minister @Theresa_May on Nerve A...,0
2,207715,"In 1961, Goodyear released a kit that allows P...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0
...,...,...,...
59995,70046,Finish Sniper Simo H盲yh盲 during the invasion o...,0
59996,189377,Nigerian Prince Scam took $110K from Kansas ma...,1
59997,93486,Is It Safe To Smoke Marijuana During Pregnancy...,0
59998,140950,Julius Caesar upon realizing that everyone in ...,0


In [ ]:
%%time
# Clean titles have less than 10 characters
df["text_clean"] = df.loc[df["text"].str.len() > 0, "text"]
df["text_clean"] = df["text_clean"].map(
    lambda x: clean_text(x) if isinstance(x, str) else x         #(lambda) Apply the fuction in each row
)

CPU times: user 40.8 s, sys: 241 ms, total: 41 s
Wall time: 51.7 s


In [ ]:
%%time
# Clean titles have less than 10 characters
df_test["text_clean"] = df_test.loc[df_test["text"].str.len() > 0, "text"]
df_test["text_clean"] = df_test["text_clean"].map(
    lambda x: clean_text(x) if isinstance(x, str) else x         #(lambda) Apply the fuction in each row
)

CPU times: user 14 s, sys: 80.5 ms, total: 14.1 s
Wall time: 14.1 s


In [ ]:
df

,id,text,label,text_clean
0,265723,A group of friends began to volunteer at a hom...,0,group friend began volunt homeless shelter nei...
1,284269,British Prime Minister @Theresa_May on Nerve A...,0,british prime minist theresa may nerv attack f...
2,207715,"In 1961, Goodyear released a kit that allows P...",0,goodyear releas kit allow ps brought heel http...
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0,happi birthday bob barker price right host lik...
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0,obama nation innoc cop unarm young black men d...
...,...,...,...,...
59995,70046,Finish Sniper Simo H盲yh盲 during the invasion o...,0,finish sniper simo yh invas finland ussr color
59996,189377,Nigerian Prince Scam took $110K from Kansas ma...,1,nigerian princ scam took kansa man year later ...
59997,93486,Is It Safe To Smoke Marijuana During Pregnancy...,0,safe smoke marijuana pregnanc surpris answer
59998,140950,Julius Caesar upon realizing that everyone in ...,0,julius caesar upon realiz everyon room knife e...


### Modeling and Splitting data
Splitting data into training and validation

In [ ]:
df_test.isna().sum()

id            0
text          0
text_clean    0
dtype: int64

In [ ]:
y = df['label']
X = df['text_clean']
X_test = df_test['text_clean']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0, train_size=0.8)
split_index = [-1 if i in X_train.index else 0 for i in X.index]
pds = PredefinedSplit(test_fold = split_index)

In [ ]:
X_train

38618    googl franc shut googl page ask user follow tw...
50372    psbattl boy watch televis first time applianc ...
12024    bulli pour super glue head believ next cut hai...
20678    ww photograph catch final moment japanes suici...
26866    new england patriot fan react woman wear eagl ...
                               ...                        
46080     pastor said santa claus jaw drop said santa real
52619    kierkegaard help us cope trump relat anxieti s...
42793    nasa look someon protect earth alien job pay s...
43751    josef mengel angel death experi jew auschwitz ...
2747     despit effort feder offici crack rampant ident...
Name: text_clean, Length: 47814, dtype: object

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(47814,)
(47814,)
(11954,)
(11954,)


# Trials

## Trials vectorizer analyzer="word"

**Trail_1**

Search: Random Search.

Classifier: Logistic Regression Classifier.

Accuracy: Score: 0.81157

private: 0.81379




---


**Trail_2**

Search: Random Search.

Classifier: XGB Classifier.

Accuracy: Score: 0.78700

private: 0.78520



---



**Trail_3**

Search: Grid Search.

Classifier: Logistic Classifier.

Accuracy: Score: 0.83771

private: 0.83929



---


 
**Trail_4**

Search: Random Search.

Classifier: AdaBoost Classifier.

Accuracy: Score: 0.68689

private: 0.68363


---


**Trail_5**

Search: Grid Search.

Classifier: Logistic Classifier.

Accuracy: Score: 0.83364

Private: 0.83590



---



---


## Trials vectorizer analyzer="char"

**Trail_1**

Search: Random Search.

Classifier: XGB Classifier.

Accuracy: Score: 0.78700

private: 0.78520




---



**Trail_2**

Search: Random Search.

Classifier: XGB Classifier.

Accuracy: Score: 0.75775

private: 0.75452





---



**Trail_3**

Search: Random Search.

Classifier: AdaBoost Classifier.

Accuracy: Score: 0.68350

private: 0.68407




---



**Trail_4**

Search: Random Search.

Classifier: Decision Tree Classifier.

Accuracy: Score: 0.66767

private: 0.67157


#Trials vectorizer analyzer="word"

At first, I will try a logistic model in the data with random search

##Trial_1
Random Search

Logistic Regression Classifier

In [ ]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l1")), ("logistic", LogisticRegression())])

params = {
    "tfidf__ngram_range": [(1, 3),(1, 2)],
        # tfidf__ngram_range points to tfidf ->ngram_range

    "tfidf__max_df": [0.5],
        # tfidf__ngram_range points to tfidf ->max_df

    "tfidf__min_df": [5],
        # tfidf__ngram_range points to tfidf ->min_df

}
#Search about best values to hyperparameters
pipe_lg_clf = RandomizedSearchCV(pipe, params, n_jobs=-1, scoring="roc_auc", cv = pds, n_iter=3)
pipe_lg_clf.fit(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 2 is smaller than n_iter=3. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


CPU times: user 3.8 s, sys: 1.83 s, total: 5.62 s
Wall time: 11.8 s


In [ ]:
print(pipe_lg_clf.best_params_)

{'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 5, 'tfidf__max_df': 0.5}


In [ ]:
pipe_lg_clf.score(X_val, y_val)

0.8787478664442014

In [ ]:
X_test

0                                                stargaz
1                                                   yeah
2         pd phoenix car thief get instruct youtub video
3                   trump accus iran one problem credibl
4                                       believ hezbollah
                              ...                       
59146                       bicycl taxi driver new delhi
59147               trump blow gop formula win hous race
59148    napoleon return exil island elba march colouris
59149                      deep alway want ballet dancer
59150         toddler miracul surviv stori fall land car
Name: text_clean, Length: 59151, dtype: object

In [ ]:
#Write the result in csv file

submission = pd.DataFrame()
submission['id'] = df_test['id']

submission['label'] = pipe_lg_clf.predict_proba(X_test)[:,1]

submission.to_csv('/content/drive/MyDrive/Colab_Notebooks/DM/Project_3/sample_submission_Trail_1.csv', index=False)

Score: 0.81157

private: 0.81379

##Trial_2
Random Search

XGBClassifier

According to the previous model result, I will change the model.

In [ ]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l2")), ("XGB", XGBClassifier())])

# define parameter space to test # runtime 19min
params = {
      "tfidf__ngram_range": [(1, 2)],
      # tfidf__ngram_range points to tfidf ->ngram_range

      "tfidf__max_df": [0.5],
      # tfidf__ngram_range points to tfidf ->max_df

      "tfidf__min_df": [5,6,7],
      # tfidf__ngram_range points to tfidf ->min_df

      "XGB__learning_rate":[0.1, 0.5, 0.01, 0.51],
      # XGB__learning_rate points to  XGB->learning_rate

      "XGB__n_estimators":[200, 150, 250]
      # XGB__n_estimators points to  XGB->n_estimators
}

#Search about best values to hyperparameters
pipe_XGB_clf = RandomizedSearchCV(pipe, params, n_jobs=-1, scoring="roc_auc", cv = pds, n_iter=3)
pipe_XGB_clf.fit(X, y)

CPU times: user 26.5 s, sys: 296 ms, total: 26.8 s
Wall time: 1min 25s


In [ ]:
print(pipe_XGB_clf.best_params_)

{'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 6, 'tfidf__max_df': 0.5, 'XGB__n_estimators': 200, 'XGB__learning_rate': 0.51}


In [ ]:
pipe_XGB_clf.score(X_val, y_val)

0.8805718380787676

In [ ]:
X_test

0                                                stargaz
1                                                   yeah
2         pd phoenix car thief get instruct youtub video
3                   trump accus iran one problem credibl
4                                       believ hezbollah
                              ...                       
59146                       bicycl taxi driver new delhi
59147               trump blow gop formula win hous race
59148    napoleon return exil island elba march colouris
59149                      deep alway want ballet dancer
59150         toddler miracul surviv stori fall land car
Name: text_clean, Length: 59151, dtype: object

In [ ]:
#Write the result in csv file

submission = pd.DataFrame()
submission['id'] = df_test['id']

submission['label'] = pipe_XGB_clf.predict_proba(X_test)[:,1]

submission.to_csv('/content/drive/MyDrive/Colab_Notebooks/DM/Project_3/sample_submission_Trail_2.csv', index=False)

Score: 0.78700

private: 0.78520

##Trial_3
Grid Search

Logistic Classifier

In [ ]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l2")), ("logistic", LogisticRegression())])

# define parameter space to test # runtime 19min
params = {
    "tfidf__ngram_range": [(1, 3)],
    # tfidf__ngram_range points to tfidf ->ngram_range

    "tfidf__max_df": [0.5],
    # tfidf__ngram_range points to tfidf ->max_df
      
    "tfidf__min_df": [5],
      # tfidf__ngram_range points to tfidf ->min_df

    "logistic__penalty":['l1', 'l2']
    # logistic__penalty points to  logistic->penalty
}

#Search about best values to hyperparameters
pipe_lg_clf = GridSearchCV(pipe, params, n_jobs= -1, scoring="roc_auc", cv = pds, verbose=1)
pipe_lg_clf.fit(X, y)


Fitting 1 folds for each of 2 candidates, totalling 2 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
1 fits failed out of a total of 2.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "

CPU times: user 6.36 s, sys: 2.85 s, total: 9.22 s
Wall time: 13.8 s


In [ ]:
print(pipe_lg_clf.best_params_)

{'logistic__penalty': 'l2', 'tfidf__max_df': 0.5, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 3)}


In [ ]:
pipe_lg_clf.score(X_val, y_val)

0.9265005398051288

In [ ]:
#Write the result in csv file

submission = pd.DataFrame()
submission['id'] = df_test['id']

submission['label'] = pipe_lg_clf.predict_proba(X_test)[:,1]

submission.to_csv('/content/drive/MyDrive/Colab_Notebooks/DM/Project_3/sample_submission_Trail_3.csv', index=False)

Score: 0.83771

private: 0.83929

##Trial_4
Random Search

AdaBoost Classifier

According to the previous model, I will try AdaBoost classifier.

In [ ]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l2")), ("AB", AdaBoostClassifier())])

# define parameter space to test # runtime 19min
params = {
    "tfidf__ngram_range": [(1, 2)],
    # tfidf__ngram_range points to tfidf ->ngram_range

    "tfidf__max_df": [0.5,1,0.6, 0.9],
    # tfidf__ngram_range points to tfidf ->max_df
      
    "tfidf__min_df": [5, 4, 6 ,8],
      # tfidf__ngram_range points to tfidf ->min_df

}

#Search about best values to hyperparameters
pipe_AB_clf = RandomizedSearchCV(pipe, params, n_jobs=-1, scoring="roc_auc", cv = pds, n_iter=3)
pipe_AB_clf.fit(X, y)


CPU times: user 10.6 s, sys: 147 ms, total: 10.8 s
Wall time: 30.7 s


In [ ]:
print(pipe_AB_clf.best_params_)

{'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 4, 'tfidf__max_df': 0.9}


In [ ]:
pipe_AB_clf.score(X_val, y_val)

0.743097150257606

In [ ]:
#Write the result in csv file

submission = pd.DataFrame()
submission['id'] = df_test['id']

submission['label'] = pipe_AB_clf.predict_proba(X_test)[:,1]

submission.to_csv('/content/drive/MyDrive/Colab_Notebooks/DM/Project_3/sample_submission_Trail_4.csv', index=False)

Score: 0.68689

private: 0.68363


##Trial_5
Grid Search

Logistic Classifier

According to the previous logistic model result and it has a large result, I will change the value of the hyperparameter to increase accuracy and the way of search.

In [ ]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l2")), ("logistic", LogisticRegression())])

# define parameter space to test # runtime 19min
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    # tfidf__ngram_range points to tfidf ->ngram_range

    "tfidf__max_df": np.arange(0.5, 1),
    # tfidf__ngram_range points to tfidf ->max_df
      
    "tfidf__min_df": np.arange(10, 30),
      # tfidf__ngram_range points to tfidf ->min_df

    "logistic__penalty":['l1', 'l2']
    # logistic__penalty points to  logistic->penalty
}

#Search about best values to hyperparameters
pipe_lg_clf = GridSearchCV(pipe, params, n_jobs= 2, scoring="roc_auc", cv = pds, verbose=1)
pipe_lg_clf.fit(X, y)


Fitting 1 folds for each of 80 candidates, totalling 80 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  Fil

CPU times: user 10.7 s, sys: 2.93 s, total: 13.6 s
Wall time: 3min 32s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
print(pipe_lg_clf.best_params_)

{'logistic__penalty': 'l2', 'tfidf__max_df': 0.5, 'tfidf__min_df': 10, 'tfidf__ngram_range': (1, 3)}


In [ ]:
pipe_lg_clf.score(X_val, y_val)

0.9144137276336228

In [ ]:
#Write the result in csv file

submission = pd.DataFrame()
submission['id'] = df_test['id']

submission['label'] = pipe_lg_clf.predict_proba(X_test)[:,1]

submission.to_csv('/content/drive/MyDrive/Colab_Notebooks/DM/Project_3/sample_submission_Trail_5.csv', index=False)

Score: 0.83364

Private: 0.83590

#Trial by using vectorizer analyzer="char"

##Trial_1
Random Search

XGB Classifier

In [ ]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="char", norm="l2")), ("XGB", XGBClassifier())])

# define parameter space to test # runtime 19min
params = {
    "tfidf__ngram_range": [(1, 3)],
    # tfidf__ngram_range points to tfidf ->ngram_range

    "tfidf__max_df": [0.5],
    # tfidf__ngram_range points to tfidf ->max_df
      
    "tfidf__min_df": [5],
      # tfidf__ngram_range points to tfidf ->min_df

}

#Search about best values to hyperparameters
pipe_XGB_clf2 = RandomizedSearchCV(pipe, params, n_jobs=-1, scoring="roc_auc", cv = pds, n_iter=3)
pipe_XGB_clf2.fit(X, y)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 1 is smaller than n_iter=3. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


CPU times: user 1min 44s, sys: 469 ms, total: 1min 45s
Wall time: 3min 7s


In [ ]:
print(pipe_XGB_clf2.best_params_)

{'tfidf__ngram_range': (1, 3), 'tfidf__min_df': 5, 'tfidf__max_df': 0.5}


In [ ]:
pipe_XGB_clf2.score(X_val, y_val)

0.7957877573099317

In [ ]:
X_test

0                                                stargaz
1                                                   yeah
2         pd phoenix car thief get instruct youtub video
3                   trump accus iran one problem credibl
4                                       believ hezbollah
                              ...                       
59146                       bicycl taxi driver new delhi
59147               trump blow gop formula win hous race
59148    napoleon return exil island elba march colouris
59149                      deep alway want ballet dancer
59150         toddler miracul surviv stori fall land car
Name: text_clean, Length: 59151, dtype: object

In [ ]:
#Write the result in csv file

submission = pd.DataFrame()
submission['id'] = df_test['id']

submission['label'] = pipe_XGB_clf.predict_proba(X_test)[:,1]

submission.to_csv('/content/drive/MyDrive/Colab_Notebooks/DM/Project_3/sample_submission_Trail_1_char.csv', index=False)

Score:  0.78700

private: 0.78520



##Trial_2
Random Search

XGB Classifier

According to the previous result, I can change the value of the hyperparameter to increase accuracy.

In [ ]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="char", norm="l2")), ("XGB", XGBClassifier())])

# define parameter space to test # runtime 19min
params = {
    "tfidf__ngram_range": [(1, 2)],
    # tfidf__ngram_range points to tfidf ->ngram_range

    "tfidf__max_df": [0.5,1,0.6, 0.9],
    # tfidf__ngram_range points to tfidf ->max_df
      
    "tfidf__min_df": [5, 4, 6 ,8],
      # tfidf__ngram_range points to tfidf ->min_df

}

#Search about best values to hyperparameters
pipe_XGB_clf2 = RandomizedSearchCV(pipe, params, n_jobs=-1, scoring="roc_auc", cv = pds, n_iter=3)
pipe_XGB_clf2.fit(X, y)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
1 fits failed out of a total of 3.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 355, in _fit
    **fit_params_steps[name],
  File "/usr/local/lib/python3.7/dist-packages/joblib/memory.py", line 34

CPU times: user 1min, sys: 332 ms, total: 1min 1s
Wall time: 2min 25s


In [ ]:
print(pipe_XGB_clf2.best_params_)

{'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 6, 'tfidf__max_df': 0.9}


In [ ]:
pipe_XGB_clf2.score(X_val, y_val)

0.7504664350637558

In [ ]:
#Write the result in csv file

submission = pd.DataFrame()
submission['id'] = df_test['id']

submission['label'] = pipe_XGB_clf2.predict_proba(X_test)[:,1]

submission.to_csv('/content/drive/MyDrive/Colab_Notebooks/DM/Project_3/sample_submission_Trail_2_char.csv', index=False)

Score:  0.75775

private: 0.75452



##Trial_3
Random Search

AdaBoost Classifier

According to the previous model, I will change the model.

In [ ]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="char", norm="l2")), ("AB", AdaBoostClassifier())])

# define parameter space to test # runtime 19min
params = {
    "tfidf__ngram_range": [(1, 2)],
    # tfidf__ngram_range points to tfidf ->ngram_range

    "tfidf__max_df": [0.5,1 ,0.6, 0.9],
    # tfidf__ngram_range points to tfidf ->max_df
      
    "tfidf__min_df": [5, 4, 6 ,8],
      # tfidf__ngram_range points to tfidf ->min_df

}

#Search about best values to hyperparameters
pipe_AB_clf2 = RandomizedSearchCV(pipe, params, n_jobs=-1, scoring="roc_auc", cv= pds, n_iter=3)
pipe_AB_clf2.fit(X, y)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
1 fits failed out of a total of 3.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 355, in _fit
    **fit_params_steps[name],
  File "/usr/local/lib/python3.7/dist-packages/joblib/memory.py", line 34

CPU times: user 32.8 s, sys: 255 ms, total: 33.1 s
Wall time: 1min 16s


In [ ]:
print(pipe_AB_clf2.best_params_)

{'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 6, 'tfidf__max_df': 0.5}


In [ ]:
pipe_AB_clf2.score(X_val, y_val)

0.6916894264753827

In [ ]:
#Write the result in csv file

submission = pd.DataFrame()
submission['id'] = df_test['id']

submission['label'] = pipe_AB_clf2.predict_proba(X_test)[:,1]

submission.to_csv('/content/drive/MyDrive/Colab_Notebooks/DM/Project_3/sample_submission_Trail_3_char.csv', index=False)

Score:  0.68350

private: 0.68407


##Trial_4
Random Search

Decision Tree Classifier

According to the previous model result, I will change the model.

In [ ]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="char", norm="l2")), ("dtc", DecisionTreeClassifier())])

params = {
    "tfidf__ngram_range": [(1, 2)],
    # tfidf__ngram_range points to tfidf ->ngram_range

    "tfidf__max_df": np.arange(0.5, 1),
    # tfidf__ngram_range points to tfidf ->max_df
      
    "tfidf__min_df": np.arange(15,30),    
      # tfidf__ngram_range points to tfidf ->min_df

    "dtc__criterion": ['gini', 'entropy'],
    # dtc__criterion points to  dtc->criterion

    "dtc__max_depth":  np.arange(10,30), #15
    # dtc__criterion points to  dtc->max_depth
}

#Search about best values to hyperparameters
pipe_dtc_clf2 = RandomizedSearchCV(pipe, params, n_jobs=-1, scoring="roc_auc", cv= pds, n_iter=5)
pipe_dtc_clf2.fit(X, y)

#{'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 22, 'tfidf__max_df': 0.5, 'dtc__max_depth': 15, 'dtc__criterion': 'entropy'}  #0.7704306370151766
#{'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 18, 'tfidf__max_df': 0.5, 'dtc__max_depth': 18, 'dtc__criterion': 'entropy'}  #0.8242552719839447

CPU times: user 27.7 s, sys: 118 ms, total: 27.8 s
Wall time: 1min 59s


In [ ]:
print(pipe_dtc_clf2.best_params_)

{'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 18, 'tfidf__max_df': 0.5, 'dtc__max_depth': 18, 'dtc__criterion': 'entropy'}


In [ ]:
pipe_dtc_clf2.score(X_val, y_val)

0.8242552719839447

In [ ]:
#Write the result in csv file

submission = pd.DataFrame()
submission['id'] = df_test['id']

submission['label'] = pipe_dtc_clf2.predict_proba(X_test)[:,1]

submission.to_csv('/content/drive/MyDrive/Colab_Notebooks/DM/Project_3/sample_submission_Trail_4_char.csv', index=False)

Score:  0.66767

private: 0.67157


#**The best solution**

##Trials vectorizer analyzer="word"

**Trail_3**

Search: Grid Search.

Classifier: Logistic Classifier.

Public score: 0.83771

Private Score: 0.83929




# Questions

### What is the difference between Character n-gram and Word n-gram? Which one tends to suffer more from the OOV issue?

Word n-gram: split the text into individual words. 

Charachters n-gram: split the text into set of Characters.

OOV:  Out Of Vocabulary

    -It is a word in test data but not founded in training data.


Word have OOV problem more than Character n-gram.

###What is the difference between stop word removal and stemming? Are these techniques language-dependent?

Stop words: These are words that are frequently used in all documents so we can remove them from documents. Stop words like is, the, and.

Stemming: These mean the same word from different forms like play, play, and playing the same word but in different forms. So we can remove replace all of them into the first form (play).


Yes, each language has different stop words and steeming.

###Is tokenization techniques language dependent? Why?

Tokenization is split text into a small unit called tokens.

Yes, Tokenization depends on language because it is possible for the same word to have the same letters, maybe different sounds but different meanings in each language.

###What is the difference between count vectorizer and tf-idf vectorizer? Would it be feasible to use all possible n-grams? If not, how should you select them?

Count vectorizer: try to learn the word vocabulary and try to create a document term matrix whitch convert a given set of strings into a frequency representation.

Tf-idf vectorizer: Term frequency-inverse document frequency that takes the importance of a word depending on how frequently it occurs in a document.


No, It is selected by using try and error and tuning hyperparameters.